In [1]:
import os
import base64
import pandas as pd

from textual_features.agent_extraction import Llm, BP_Metrics_Getter

[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\LENOVO\anaconda3\envs\bp_digitalization\Lib\s
[nltk_data]     ite-packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
from textual_features.prompts_and_parsers import Llm_Extraction_Prompt

In [3]:
from azure_authentication.customized_azure_login import CredentialFactory

In [4]:
os.chdir("../")
CWD = os.getcwd()

data_dir = os.path.join(CWD, 'data')

#Specify mode (working with a sample or all the files?)
sample_mode = False 
sample_size = 50

# specify file path
INPUT_FILE_PATH = os.path.join(CWD, "data", "proc", "building_plans_sample", "test_images", "bp_text.json")
METADATA_PATH = os.path.join(CWD, "data", "proc", "building_plans", "metadata","building_plans_metadata.csv")

# specify relevant column names
ID_COLUMN='filename'
TEXT_COLUMN='content'

# read in data
bp_text = pd.read_json(INPUT_FILE_PATH)
metadata_df = pd.read_csv(METADATA_PATH)

In [5]:
metadata_bps = metadata_df[metadata_df['Planart'].isin(['qualifizierter Bebauungsplan', 'einfacher Bebauungsplan', 'vorhabenbezogener Bebauungsplan'])]

In [6]:
bp_text['id'] = bp_text['filename'].str.extract(r'(\d+)_').astype(int)

In [7]:
#input_df = metadata_bps.merge(bp_text)

In [8]:
input_df = bp_text

In [9]:
# Recommendation: Configure your own authentication workflow with environment variables, see the description at
# https://github.com/soda-lmu/azure-auth-helper-python/blob/main/AuthenticationWorkflowSetup.md
credential = CredentialFactory().select_credential()
token_provider = credential.get_login_token_to_azure_cognitive_services()

print("Instantiate Azure OpenAI Client")

Instantiate Azure OpenAI Client


In [10]:
if sample_mode:
    
    run_data = input_df.sample(sample_size, random_state=15)

else: 
 
    run_data = input_df

In [11]:
llm = Llm(model_name="gpt-4-1106-preview")

NameError: name 'asyncio' is not defined

In [16]:
getter = BP_Metrics_Getter(llm = llm, llm_single_prompt = Llm_Extraction_Prompt())

NameError: name 'llm' is not defined

In [21]:
nest_asyncio.apply()

extraction_results = await getter._bound_get_emissions_from_raw_text(run_data['content'][0])

Current Time: 10:33:45 ( 0.05705804166694482 minutes since process started.)
LLM query execution time: 5.4869639000389725 seconds (8.910447800066322 seconds since initiating ValueRetrieverPipeline).


In [25]:
parsed_extractions = getter._parse_to_classes_llm_output(extraction_results)

In [26]:
extraction_df = pd.concat(parsed_extractions)

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

In [17]:
OUTPUT_FLOODING_FILE_PATH = os.path.join("data", "proc", "building_plans_sample", "features",  "test_images_flooding_data_extraction.csv")
OUTPUT_EXTRACTIONS_FILE_PATH = os.path.join("data", "proc", "building_plans_sample", "features",  "test_images_info_data_extraction.csv")


extraction_df.to_csv(OUTPUT_EXTRACTIONS_FILE_PATH)
flooding_df.to_csv(OUTPUT_FLOODING_FILE_PATH)